# Lanchain OpenAI Setup

In [16]:
import langchain
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

# Import chat templates
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Caching
from langchain.cache import InMemoryCache
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache

In [ ]:
cacheType = 'in_memory'

if cacheType == 'in_memory':
    set_llm_cache(InMemoryCache())
elif cacheType == 'sqlite':
    set_llm_cache(SQLiteCache(database_path=".langchain.db"))

# Set OpenAI API key and create LLM and Chat LLM. Note that key can be stored in a separate file or as an environment variable. Refer to docs.
api_key = open('./openai_key.txt').read()
llm = OpenAI(openai_api_key=api_key)
chat = ChatOpenAI(openai_api_key=api_key)

## Few Shot Prompt Template

- Sometimes it's easier to give the LLM a few examples of input/output pairs before sending you main request
- Allows LLM to "learn" the pattern you are looking for and may lead to better results.
- LangChain recommends building a history of Huma and AI message inputs.

### Use Case

AI Bot that is able to simplify legal wording into simpler phrases and terms

We can provide sample prompts liks this:
- legal_term = The provisions herein shall be severable, and if any provision or portion thereof is deemed invalid, illegal, or unenforceable by a court of competent jurisdiction, the remaining provisions or portions thereof shall remain in full force and effect to the maximum extent permitted by law.
- Sample response = The rules in this agreement can be separated. If a court decides that one rule or part of it is not valid, illegal, or cannot be enforced, the other rules will still apply and be enforced as much as they can under the law.

In [27]:
# system setup
template = "You are a helpful assistant that translates complex legal terms into plain and understandable terms."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template = "{legal_text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [28]:
# Shot Prompt
sample_input_1="The provisions herein shall be severable, and if any provision or portion thereof is deemed invalid, illegal, or unenforceable by a court of competent jurisdiction, the remaining provisions or portions thereof shall remain in full force and effect to the maximum extent permitted by law."
sample_output_1 = "Sample response = The rules in this agreement can be separated. If a court decides that one rule or part of it is not valid, illegal, or cannot be enforced, the other rules will still apply and be enforced as much as they can under the law."

sample_human_prompt_1 = HumanMessagePromptTemplate.from_template(sample_input_1)
sample_ai_output_1 = AIMessagePromptTemplate.from_template(sample_output_1)

In [29]:
# Build request with few shot prompt

chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt, 
    sample_human_prompt_1, sample_ai_output_1, 
    human_message_prompt])

In [30]:
chat_prompt.input_variables

['legal_text']

In [31]:
test_legal = "The grantor, being the fee simple owner of the real property herein described, conveys and warrants to the grantee, his heirs and assigns, all of the grantor's right, title, and interest in and to the said property, subject to all existing encumbrances, liens, and easements, as recorded in the official records of the county, and any applicable covenants, conditions, and restrictions affecting the property, in consideration of the sum of [purchase price] paid by the grantee."
request = chat_prompt.format_prompt(legal_text = test_legal).to_messages()

In [33]:
result = chat(request)

In [34]:
print(result.content)

The person who owns the property, known as the grantor, is transferring all their rights, ownership, and interest in the property to the recipient, known as the grantee. This transfer is subject to any existing debts, claims, or rights that have been officially recorded, as well as any rules or restrictions that apply to the property. The grantor is doing this in exchange for a specific amount of money that the grantee has paid.
